In [1]:
# import pandas

# import modin.pandas as pd
# from modin.config import Engine
# Engine.put("dask") 

In [2]:
import os
import daal4py as d4p
from xgboost import XGBClassifier
import time
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# pio.renderers.default='notebook_connected' 
# intel_pal, color=['#0071C5','#FCBB13'], ['#7AB5E1','#FCE7B2']
temp=dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), height=500, width=1000))

In [3]:
# from distributed import Client
# client = Client()

# data = pd.read_csv("dataset.csv")

data = pd.read_csv("50k_dataset.csv")
data = data.drop('Index', axis=1)
print("Total rows in the DataFrame:", len(data))

print(data.dtypes)
display(data.isna().sum())

Total rows in the DataFrame: 5956842
pH                        float64
Iron                      float64
Nitrate                   float64
Chloride                  float64
Lead                      float64
Zinc                      float64
Color                      object
Turbidity                 float64
Fluoride                  float64
Copper                    float64
Odor                      float64
Sulfate                   float64
Conductivity              float64
Chlorine                  float64
Manganese                 float64
Total Dissolved Solids    float64
Source                     object
Water Temperature         float64
Air Temperature           float64
Month                      object
Day                       float64
Time of Day               float64
Target                      int64
dtype: object


pH                        116054
Iron                       39753
Nitrate                   105725
Chloride                  175531
Lead                       26909
Zinc                      156126
Color                       5739
Turbidity                  49815
Fluoride                  189156
Copper                    199402
Odor                      178891
Sulfate                   197418
Conductivity              163861
Chlorine                   57825
Manganese                 109583
Total Dissolved Solids      1670
Source                     88262
Water Temperature         168233
Air Temperature            29728
Month                      95668
Day                        99603
Time of Day               114519
Target                         0
dtype: int64

In [4]:
# handle missing values
start_fill = time.time()
for column in data.columns:
    if data[column].dtype == 'object':
        mode = data[column].mode().iloc[0]
        data[column].fillna(mode, inplace=True)
    else:
        median = data[column].median()
        data[column].fillna(median, inplace=True)

end_fill = time.time()
print("fill missing value took: {:.2f} s".format(end_fill - start_fill))

# drop duplicated rows
data = data.drop_duplicates()

# transform `Color` to number
color_labels, color_unique_values = pd.factorize(data['Color'])
data['Color'] = color_labels
# transform `Source` to number
source_labels, source_unique_values = pd.factorize(data['Source'])
data['Source'] = source_labels

fill missing value took: 3.75 s


In [5]:
display(data.isna().sum())
missing = data.isna().sum().sum()
duplicates = data.duplicated().sum()
print("\nThere are {:,.0f} missing values in the data.".format(missing))
print("There are {:,.0f} duplicate records in the data.".format(duplicates))

display(data.head())

pH                        0
Iron                      0
Nitrate                   0
Chloride                  0
Lead                      0
Zinc                      0
Color                     0
Turbidity                 0
Fluoride                  0
Copper                    0
Odor                      0
Sulfate                   0
Conductivity              0
Chlorine                  0
Manganese                 0
Total Dissolved Solids    0
Source                    0
Water Temperature         0
Air Temperature           0
Month                     0
Day                       0
Time of Day               0
Target                    0
dtype: int64


There are 0 missing values in the data.
There are 0 duplicate records in the data.


,pH,Iron,Nitrate,Chloride,Lead,Zinc,Color,Turbidity,Fluoride,Copper,...,Chlorine,Manganese,Total Dissolved Solids,Source,Water Temperature,Air Temperature,Month,Day,Time of Day,Target
0,8.332988,0.000083,8.605777,122.799772,3.713298e-52,3.434827,0,0.022683,0.607283,0.144599,...,3.708178,2.269945e-15,332.118789,0,16.444276,43.493324,January,29.0,4.0,0
1,6.917863,0.000081,3.734167,227.029851,7.849262e-94,1.245317,1,0.019007,0.622874,0.437835,...,3.292038,8.024076e-07,284.641984,1,15.348981,71.220586,November,26.0,16.0,0
2,5.443762,0.020106,3.816994,230.995630,5.286616e-76,0.528280,2,0.319956,0.423423,0.431588,...,3.560224,7.007989e-02,570.054094,2,11.643467,44.891330,January,31.0,8.0,0
3,7.955339,0.143988,8.224944,178.129940,3.997118e-176,4.027879,3,0.166319,0.208454,0.239451,...,3.516907,2.468295e-02,100.043838,3,10.092392,60.843233,April,1.0,21.0,0
4,8.091909,0.002167,9.925788,186.540872,4.171069e-132,3.807511,2,0.004867,0.222912,0.616574,...,3.177849,3.296139e-03,168.075545,4,15.249416,69.336671,June,29.0,7.0,0


In [6]:
def display_stats():
    
    """
    Function to display descriptive statistics of numerical variables,
    includes skewness & kurtosis.   
    """
    
    df = data.describe()
    skewness = pd.DataFrame(data.skew(numeric_only=True), columns=['skewness']).T
    kurtosis = pd.DataFrame(data.kurt(numeric_only=True), columns=['kurtosis']).T
    df = pd.concat([df, skewness, kurtosis], axis=0)
    display(df.style.format('{:,.3f}').background_gradient(subset=(df.index[1:],df.columns[:]), cmap='GnBu'))

display_stats()

,pH,Iron,Nitrate,Chloride,Lead,Zinc,Color,Turbidity,Fluoride,Copper,Odor,Sulfate,Conductivity,Chlorine,Manganese,Total Dissolved Solids,Source,Water Temperature,Air Temperature,Day,Time of Day,Target
count,"5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000","5,844,805.000"
mean,7.446,0.127,6.159,184.017,0.001,1.538,1.697,0.518,0.958,0.510,1.802,145.674,424.225,3.255,0.107,267.126,3.447,19.053,60.002,15.737,11.514,0.302
std,0.879,0.478,3.227,67.373,0.032,1.527,1.399,0.922,0.812,0.587,1.053,67.632,187.380,0.729,0.472,155.814,2.314,11.210,18.054,8.720,6.855,0.459
min,1.057,0.000,0.286,23.639,0.000,0.000,0.000,0.000,0.000,0.000,0.011,11.941,10.600,0.902,0.000,0.010,0.000,0.666,-33.871,1.000,0.000,0.000
25%,6.907,0.000,4.003,139.366,0.000,0.429,0.000,0.040,0.387,0.135,0.920,99.054,289.715,2.750,0.000,132.963,1.000,11.495,47.864,8.000,6.000,0.000
50%,7.450,0.002,5.604,176.018,0.000,1.082,2.000,0.210,0.775,0.348,1.774,134.649,397.081,3.210,0.001,265.888,3.000,16.444,59.997,16.000,12.000,0.000
75%,8.002,0.054,7.624,216.445,0.000,2.186,3.000,0.619,1.317,0.685,2.626,179.680,528.376,3.699,0.016,398.421,5.000,23.536,72.139,23.000,17.000,1.000
max,12.911,19.353,96.391,"1,507.310",5.844,28.369,4.000,23.715,14.646,12.075,4.142,"1,434.587","2,271.632",12.567,23.741,579.800,7.000,297.309,152.124,31.000,23.000,1.000
skewness,-0.089,8.518,2.283,1.632,43.802,2.076,0.087,4.561,2.180,3.165,0.173,1.776,0.918,0.740,8.834,0.054,0.016,2.031,-0.001,0.005,-0.005,0.862
kurtosis,1.519,107.753,12.214,6.295,"2,848.937",7.741,-1.445,31.782,8.825,16.503,-0.892,7.851,1.326,1.854,116.619,-1.119,-1.253,8.054,0.013,-1.161,-1.168,-1.258


In [7]:
# show target distribution
target=data['Target'].value_counts(normalize=True)
print(target)

0    0.697833
1    0.302167
Name: Target, dtype: float64


In [ ]:
cat_cols,float_cols=[],['Target']
for col in data.columns:
    if data[col].value_counts().count()<10:
        cat_cols.append(col)
    else:
        float_cols.append(col)
        
plot_df=data[float_cols]

# do not display distribution with column `Month` and `Day`
columns_to_drop = ['Month', 'Day']
plot_df.drop([columns_to_drop], axis=1, inplace=True)

fig, ax = plt.subplots(9,2, figsize=(10,20))
fig.suptitle('Distribution of Numerical Variables',fontsize=16)
row=0
col=[0,1]*9

for i, column in enumerate(plot_df.columns[1:-1]):
    if (i!=0)&(i%2==0):
        row+=1
    
    sns.kdeplot(
        x=column, hue='Target', hue_order=[1, 0],
        label=['State 1', 'State 0'], data=plot_df,
        fill=True, linewidth=2.5, legend=False, ax=ax[row, col[i]]
    )
    
    ax[row,col[i]].tick_params(left=False, bottom=False)
    ax[row,col[i]].set(title='\n\n{}'.format(column), xlabel='', ylabel=('Density' if i%2==0 else ''))

handles, _ = ax[0,0].get_legend_handles_labels() 
fig.legend(labels=['State 1','State 0'], handles=reversed(handles), ncol=2, bbox_to_anchor=(0.18, 0.99))
sns.despine(bottom=True, trim=True)
plt.tight_layout()

/home/u188545/tmp/ipykernel_3655931/2601483805.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plot_df.drop(columns_to_drop, axis=1, inplace=True)


In [ ]:
fig=make_subplots(rows=9,cols=2, subplot_titles=float_cols[1:-3])
col=[1,2]*9
row=0
pal=sns.color_palette("GnBu",20).as_hex()[9:][::3]

for i, column in enumerate(data[float_cols].columns[1:-3]):
    if i%2 == 0:
        row += 1
    df = pd.concat([data[column], data['Target']], axis=1)

    df['bins'] = pd.cut(df[column], 1000)
    df['mean'] = df.bins.apply(lambda x: x.mid)

    df = df.groupby('mean')[[column,'Target']].transform('mean')
    df = df.drop_duplicates(subset=[column]).sort_values(by=column)
    
    fig.add_trace(go.Scatter(x=df[column], y=df.Target, name=column,
                             marker_color=pal[i % len(pal)], showlegend=False),
                  row=row, col=col[i])
    fig.update_xaxes(zeroline=False, row=row, col=col[i])
    if i%2 == 0:
        fig.update_yaxes(title='Target Probabilitiy',row=row,col=col[i]) 
fig.update_layout(template=temp, title='Feature Relationships with Target', hovermode="x unified",height=1700,width=1200)
fig.show()